In [2]:
import pandas as pd
pd.set_option("display.max_rows", None)


In [3]:
att_data = pd.read_csv("attestation.csv")
att_data

,cycle,env,func_call
0,13718035263,linux,keystone_create_enclave
1,13718035343,linux,calculate_required_pages
2,13718035471,linux,create_enclave
3,13718237525,linux,epm_init
4,13718237589,linux,init_free_pages
5,13718243493,linux,get_free_page.part.0
6,13718244655,linux,keystone_rtld_init_runtim
7,13718760831,linux,rtld_vm_mmap
8,13718760961,linux,epm_alloc_rt_page
9,13718761071,linux,epm_alloc_page


In [4]:
linux_paging = att_data[:73] # boot?
linux_paging

,cycle,env,func_call
0,13718035263,linux,keystone_create_enclave
1,13718035343,linux,calculate_required_pages
2,13718035471,linux,create_enclave
3,13718237525,linux,epm_init
4,13718237589,linux,init_free_pages
5,13718243493,linux,get_free_page.part.0
6,13718244655,linux,keystone_rtld_init_runtim
7,13718760831,linux,rtld_vm_mmap
8,13718760961,linux,epm_alloc_rt_page
9,13718761071,linux,epm_alloc_page


In [12]:
att_data[84:86] # allocate eid

,cycle,env,func_call
84,13719093240,sm,encl_alloc_idx
85,13719093490,sm,pmp_set_global


In [13]:
att_data[85:89] # set PMP

,cycle,env,func_call
85,13719093490,sm,pmp_set_global
86,13719093572,sm,send_and_sync_pmp_ipi
87,13719094043,sm,pmp_set
88,13719094529,sm,init_encl_pgtable


In [16]:
att_data[88:98] # initialize and verify enclave memory layout

,cycle,env,func_call
88,13719094529,sm,init_encl_pgtable
89,13719094828,sm,init_encl_pgtable
90,13719095103,sm,init_encl_pgtable
91,13719105333,sm,init_encl_pgtable
92,13719126571,sm,init_encl_pgtable
93,13719133885,sm,init_encl_pgtable
94,13719141143,sm,init_encl_pgtable
95,13719141283,sm,init_encl_pgtable
96,13719155027,sm,init_encl_pgtable
97,13719162409,sm,hash_enclave


In [9]:
att_data[96:98] #  initialize enclave metadata

,cycle,env,func_call
96,13719155027,sm,init_encl_pgtable
97,13719162409,sm,hash_enclave


In [18]:
att_data[97:125] #  prepare hash and signature for attestation

,cycle,env,func_call
97,13719162409,sm,hash_enclave
98,13719162526,sm,hash_init
99,13719162844,sm,hash_epm
100,13719163087,sm,hash_epm
101,13719163311,sm,hash_epm
102,13719163503,sm,hash_extend
103,13719163739,sm,hash_extend_page
104,13723547061,sm,hash_extend_page
105,13727860574,sm,hash_epm
106,13727862823,sm,hash_extend


In [22]:
att_data[124:128] # copy hash+sig to host

,cycle,env,func_call
124,13767423801,sm,copy_word_to_host
125,13767423936,sm,detect_region_overlap_ato
126,13767424046,sm,detect_region_overlap
127,13767492288,linux,enclave_idr_alloc


In [25]:
att_data[127:130] # end of create enclave

,cycle,env,func_call
127,13767492288,linux,enclave_idr_alloc
128,13767538995,linux,keystone_mmap
129,13767547891,linux,keystone_ioctl


In [28]:
rest = att_data[130:]
rest

,cycle,env,func_call
130,13767548582,linux,keystone_run_enclave
131,13767548692,linux,get_enclave_by_id
132,13767550383,sm,mcall_sm_run_enclave
133,13767550444,sm,get_host_satp
134,13767550678,sm,run_enclave
135,13767551072,sm,swap_prev_state
136,13767551813,sm,swap_prev_mepc
137,13767551873,sm,swap_prev_stvec
138,13767552039,sm,pmp_set
139,13767552658,sm,osm_pmp_set


In [33]:
att_data[130:148] # runtime setup

,cycle,env,func_call
130,13767548582,linux,keystone_run_enclave
131,13767548692,linux,get_enclave_by_id
132,13767550383,sm,mcall_sm_run_enclave
133,13767550444,sm,get_host_satp
134,13767550678,sm,run_enclave
135,13767551072,sm,swap_prev_state
136,13767551813,sm,swap_prev_mepc
137,13767551873,sm,swap_prev_stvec
138,13767552039,sm,pmp_set
139,13767552658,sm,osm_pmp_set


In [36]:
att_data[147:168] # run in user mode until context switch

,cycle,env,func_call
147,13767555407,user,unknown
148,13767555559,user,unknown
149,13767555561,user,unknown
150,13767555616,user,unknown
151,13767555708,user,unknown
152,13767555710,user,unknown
153,13767555763,user,unknown
154,13767555766,user,unknown
155,13767555768,user,unknown
156,13767555770,user,unknown


In [40]:
att_data[167:]

,cycle,env,func_call
167,13767556041,runtime,encl_trap_handler
168,13767556729,runtime,handle_syscall
169,13767557263,sm,mcall_sm_attest_enclave
170,13767557317,sm,attest_enclave
171,13767557627,sm,get_phys_addr
172,13767557883,sm,copy_from_enclave
173,13767558158,sm,pmp_get_addr
174,13767558288,sm,pmp_get_addr
175,13767558360,sm,pmp_get_size
176,13767559886,sm,sm_sign
